In [1]:
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = solutions.pose

In [3]:
chosen_indexes = [
    0, # NOSE
    11, # LEFT_SHOULDER
    12, # RIGHT_SHOULDER
    13, # LEFT_ELBOW
    14, # RIGHT_ELBOW
    15, # LEFT_WRIST
    16, # RIGHT_WRIST
    19, # LEFT_INDEX
    20, # RIGHT_INDEX
    23, # LEFT_HIP
    24, # RIGHT_HIP
    25, # LEFT_KNEE
    26, # RIGHT_KNEE
    27, # LEFT_ANKLE
    28, # RIGHT_ANKLE
    31, # LEFT_FOOT_INDEX
    32, # RIGHT_FOOT_INDEX
]

In [4]:
names = [
    'NOSE',
    'LEFT_SHOULDER',
    'RIGHT_SHOULDER',
    'LEFT_ELBOW',
    'RIGHT_ELBOW',
    'LEFT_WRIST',
    'RIGHT_WRIST',
    'LEFT_INDEX',
    'RIGHT_INDEX',
    'LEFT_HIP',
    'RIGHT_HIP',
    'LEFT_KNEE',
    'RIGHT_KNEE',
    'LEFT_ANKLE',
    'RIGHT_ANKLE',
    'LEFT_FOOT_INDEX',
    'RIGHT_FOOT_INDEX',
    'NECK'
]

chars = ['x', 'y', 'z']

In [5]:
def generate_connections(indexes):
    """

    """

    mapping = {}
    connections = set()
    
    # Create suitable mapping
    for i, landmark in enumerate(indexes):
        mapping[landmark] = i

    # Generate custom connections
    for connection in mp_pose.POSE_CONNECTIONS:
        item1, item2 = connection
        if item1 in indexes and item2 in indexes:
            connections.add(
                (
                    mapping[item1],
                    mapping[item2]
                )
            )
    
    return connections

In [6]:
custom_connections = generate_connections(chosen_indexes)

In [7]:
def landmark2array(landmark):
    return np.array(
        [
            landmark.x,
            landmark.y,
            landmark.z,
            landmark.visibility
        ]
    )

def array2landmark(array):
    return landmark_pb2.NormalizedLandmark(
        x=array[0],
        y=array[1],
        z=array[2],
        visibility=array[3]
    )

def get_custom_landmarks(indexes, landmarks):
    """
    
    """
    # Create customize landmarks list
    custom_landmarks = landmark_pb2.NormalizedLandmarkList()

    # Extend list by chosen landmarks
    custom_landmarks.landmark.extend(
        [landmarks.landmark[index] for index in indexes])

    # Calculate the coordinates of neck landmark
    left_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER])
    right_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER])
    neck = np.mean([left_shoulder, right_shoulder], axis=0)
    neck_landmark = array2landmark(neck)

    # Add neck landmark to custom list
    custom_landmarks.landmark.add().CopyFrom(neck_landmark)

    return custom_landmarks

In [8]:
# source posibilities: path, int
# source = r'C:/Users/nemet/Desktop/XPC_2023/MVI_7011.mp4'
source = r'C:/Users/nemet/Desktop/XPC_2023/test8.mp4'
# source = 0

# background bool: Flase, True
background = True

# output posibilities: 'landmarks', 'mask', None
output = 'landmarks'

In [15]:
column_names = ['timestamp'] + [name.lower() + '_' + char for name in names for char in chars]

data = pd.DataFrame(columns = column_names, dtype = float)

time = 0
cap = cv2.VideoCapture(source)

# Setup MediaPipe instance
with mp_pose.Pose(
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    enable_segmentation=True
) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        image_shape = image.shape


        # Camera condition -> if selfie or web camera is chosen then flip image
        if source == 0:
            image = cv2.flip(image, 1)
    
        # Recolor image for image processing
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Image MediaPipe processing -> detection
        results = pose.process(image)

        # Recolor back to BGR for visualization
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # create a fake background
        fake_background = np.zeros(shape=image_shape, dtype='uint8')
        # use a fake background if necessary
        if not background:
            image = fake_background

        # Image processing for different outputs
        if output == 'landmarks':
            default_landmarks = results.pose_landmarks

            if default_landmarks:
                time += 1

                # Get custom landmarks and create new connection
                custom_landmarks = get_custom_landmarks(chosen_indexes, default_landmarks)
                custom_connections.add((0, len(custom_landmarks.landmark) - 1))

                # Prepare a single record storage
                record = np.array([time])

                for landmark in custom_landmarks.landmark:
                    # Extract pose landmarks coordinates and store as array
                    coordinates = landmark2array(landmark)[:3]
                    record = np.concatenate(
                        [
                            record,
                            coordinates
                        ]
                    )

                # Save pose landmarks coordinates in time as DataFrame
                data = pd.concat([data, pd.DataFrame([record], columns = column_names)], ignore_index=True)

                # Draw customize landmarks on image
                mp_drawing.draw_landmarks(
                    image,
                    landmark_list=custom_landmarks,
                    connections=custom_connections,
                )


        elif output == 'mask':
            image = results.segmentation_mask
            if image is None:
                image = fake_background


        cv2.imshow(f'background: {background}, output: {output}', image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

        if not cap.isOpened():
            exit()
            
cap.release()
cv2.destroyAllWindows()

In [13]:
data

,timestamp,nose_x,nose_y,nose_z,left_shoulder_x,left_shoulder_y,left_shoulder_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,...,right_ankle_z,left_foot_index_x,left_foot_index_y,left_foot_index_z,right_foot_index_x,right_foot_index_y,right_foot_index_z,neck_x,neck_y,neck_z
0,1.0,0.476585,0.283651,-0.226193,0.593176,0.309569,0.067215,0.386084,0.316786,0.058415,...,-0.161846,0.645625,0.776603,-0.355386,0.405638,0.778760,-0.356290,0.489630,0.313177,0.062815
1,2.0,0.477664,0.253810,-0.114875,0.592562,0.309748,0.126231,0.386703,0.315315,0.088879,...,-0.141608,0.645534,0.777155,-0.351701,0.403889,0.779085,-0.328905,0.489633,0.312531,0.107555
2,3.0,0.477114,0.275805,-0.157181,0.592548,0.309754,0.113585,0.387788,0.315149,0.082285,...,-0.132630,0.645224,0.777289,-0.341084,0.403436,0.779273,-0.306473,0.490168,0.312452,0.097935
3,4.0,0.476946,0.276480,-0.170652,0.592321,0.309597,0.107555,0.388114,0.314668,0.075961,...,-0.136069,0.644564,0.777287,-0.341190,0.403018,0.779851,-0.314975,0.490218,0.312132,0.091758
4,5.0,0.476950,0.275201,-0.172617,0.592113,0.309343,0.100778,0.388204,0.313456,0.073906,...,-0.137029,0.644306,0.777292,-0.340825,0.403191,0.780147,-0.319415,0.490158,0.311399,0.087342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,100.0,0.461384,0.233444,-0.175021,0.565282,0.272218,0.100056,0.363487,0.273823,0.082269,...,-0.001560,0.646281,0.776346,-0.218191,0.399468,0.782699,-0.176842,0.464385,0.273021,0.091162
100,101.0,0.458670,0.236419,-0.171340,0.564185,0.272984,0.102781,0.362840,0.273890,0.082210,...,-0.015634,0.646338,0.776148,-0.247930,0.398572,0.782134,-0.189016,0.463512,0.273437,0.092495
101,102.0,0.455707,0.236650,-0.171342,0.561215,0.273585,0.108101,0.361069,0.274166,0.084038,...,-0.024386,0.646593,0.775309,-0.268993,0.397043,0.781665,-0.200372,0.461142,0.273876,0.096069
102,103.0,0.452793,0.237521,-0.161045,0.560748,0.274565,0.108344,0.361102,0.274471,0.089002,...,-0.048421,0.646719,0.774481,-0.264984,0.398860,0.781896,-0.226126,0.460925,0.274518,0.098673
